In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dr2_boronia.csv')

In [3]:
class DrAudit:
    
    def __init__(self, df):
        self.df = df
    
    def study_period(self):
        # study period
        dates = pd.to_datetime(df['admission_date'])
        earliest = dates.min()
        latest = dates.max()
        return earliest, latest
    
    def num_patients(self):
        # number of patients
        return len(df['ur'].unique())
    
    def num_eyes(self):
        # number of eyes
        return len(df['eye_id'].unique())
    
    def total_visits(self):
        # total visits
        return len(df)
    
    def initiation_drug(self, id_number):
        '''
        Input: patient id_number corresponding to dataframe.
        Output: string of initiation drug for that patient.
        '''
        pdf = self.df[self.df['eye_id'] == id_number]
        pdf.dropna(subset=['Drug'], inplace=True)
        pdf['admission_date'] = pd.to_datetime(pdf['admission_date'])
        pdf = pdf.sort_values(by=['admission_date'])
        return pdf['Drug'].iloc[0]
    
    def initiation_all(self):
        '''
        Input: dataframe of all patients.
        Output: list of all initiation drugs.
        '''
        names = ['Lucentis', 'Eylea', 'Avastin']
        drugs = []
        self.df.dropna(subset=['Drug'], inplace=True)
        id_list = df['eye_id'].unique()
        for eye in id_list:
            drug = self.initiation_drug(eye)
            if drug in names:
                drugs.append(drug)
        luc_int = np.round(drugs.count('Lucentis') / len(drugs), 3)
        eyl_int = np.round(drugs.count('Eylea') / len(drugs), 3)
        av_int = np.round(drugs.count('Avastin') / len(drugs), 3)
        return luc_int, eyl_int, av_int
    
    def interval_column(self, pdf): # THIS NEEDS TO GO IN THE DF EDITING CLASS
        '''
        Input: patient dataframe.
        Output: patient dataframe with additional column for interval length.
        '''
        pdf.dropna(subset=['admission_date'], inplace=True)
        pdf['admission_date'] = pd.to_datetime(pdf['admission_date'])
        pdf = pdf.sort_values(by=['admission_date'])
        dates = pdf['admission_date'].reset_index(drop=True)
        intervals = [0]
        for i in range(len(dates)-1):
            initial, current = dates[i], dates[i+1]
            interval = int((current - initial).days / 7)
            intervals.append(interval)
        pdf['Interval'] = intervals
        return pdf.reset_index(drop=True)
    
    def interval_df(self): # ALSO NEEDS TO GO IN DF EDITING CLASS
        '''
        Input: whole dataframe.
        Output: whole dataframe with additional column for interval length.
        '''
        id_list = self.df['eye_id'].unique()
        frames = []
        for eye in id_list:
            pdf = self.df[self.df['eye_id'] == eye]
            pdf = self.interval_column(pdf)
            frames.append(pdf)
        return pd.concat(frames)

    def irf_extred(self):
        # this function counts the irf values when extending and reducing
        df = self.interval_df()
        df.dropna(subset=['irf'], inplace=True)
        df = df[df['irf'] < 1000]
        id_list = df['eye_id'].unique()
        irf_extend = []
        irf_reduce = []
        irf_maintain = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            for i in range(len(pdf)-1):
                if pdf['Interval'].iloc[i+1] > pdf['Interval'].iloc[i]:
                    irf_extend.append(pdf['irf'].iloc[i+1])
                elif pdf['Interval'].iloc[i+1] < pdf['Interval'].iloc[i]:
                    irf_reduce.append(pdf['irf'].iloc[i+1])    
                else:
                    irf_maintain.append(pdf['irf'].iloc[i+1])   
        return np.mean(irf_extend), np.mean(irf_reduce), np.mean(irf_maintain)
    
    # this function counts the srf values when extending and reducing
    def srf_extred(self):
        df = self.interval_df()
        df.dropna(subset=['srf'], inplace=True)
        df = df[df['srf'] < 1000]
        id_list = df['eye_id'].unique()
        srf_extend = []
        srf_reduce = []
        srf_maintain = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            for i in range(len(pdf)-1):
                if pdf['Interval'].iloc[i+1] > pdf['Interval'].iloc[i]:
                    srf_extend.append(pdf['srf'].iloc[i+1])
                elif pdf['Interval'].iloc[i+1] < pdf['Interval'].iloc[i]:
                    srf_reduce.append(pdf['srf'].iloc[i+1])    
                else:
                    srf_maintain.append(pdf['srf'].iloc[i+1])   
        return np.mean(srf_extend), np.mean(srf_reduce), np.mean(srf_maintain)

In [4]:
class Results(DrAudit):
    
    def __init__(self, df):
        self.df = df
        
    def results_list(self):
        results_list = ['num_patients', 
                        'num_eyes', 'total_visits', 'initiation_lucentis',
                        'initiation_eylea', 'initiation_avastin',
                        'irf_extend', 'irf_reduce', 'irf_maintain',
                        'srf_extend', 'srf_reduce', 'srf_maintain',]
        return results_list
    
    def get_results(self):
        rlst = []
        rlst.append(DrAudit.num_patients(self))
        rlst.append(DrAudit.num_eyes(self))
        rlst.append(DrAudit.total_visits(self))
        luc_int, eyl_int, av_int = DrAudit.initiation_all(self)
        rlst.append(luc_int)
        rlst.append(eyl_int)
        rlst.append(av_int)
        irf_extend, irf_reduce, irf_maintain = DrAudit.irf_extred(self)
        srf_extend, srf_reduce, srf_maintain = DrAudit.srf_extred(self)
        rlst.append(irf_extend)
        rlst.append(irf_reduce)
        rlst.append(irf_maintain)
        rlst.append(srf_extend)
        rlst.append(srf_reduce)
        rlst.append(srf_maintain)
        return rlst
    
    def create_table(self):
        lst = self.get_results()
        results_list = self.results_list()
        return pd.DataFrame(lst, index = results_list, columns=['Dr2'])

In [5]:
table1 = Results(df)
to_upload = table1.create_table()

In [6]:
to_upload

,Dr2
num_patients,374.000000
num_eyes,746.000000
total_visits,24048.000000
initiation_lucentis,0.616000
initiation_eylea,0.367000
initiation_avastin,0.017000
irf_extend,56.660828
irf_reduce,55.090423
irf_maintain,51.162084
srf_extend,35.384565


In [99]:
to_upload.to_csv('/home/jupyter/charliemacuject/pharma_reports/toupload.csv')

# Alex Tan

In [10]:
# FLUID STATE AT TIME OF SWITCHES
df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dr2_boronia.csv')

def patient_cutoff(df):
    id_list = df['eye_id'].unique()
    cutoff = pd.to_datetime('01/01/2016')
    frames = []
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        pdf['admission_date'] = pd.to_datetime(pdf['admission_date'])
        pdf.sort_values(by=['admission_date'], inplace=True)
        if pdf['admission_date'].iloc[0] > cutoff:
            frames.append(pdf)
    return pd.concat(frames)

In [2]:
df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dr2_boronia_2016.csv')
df['admission_date'] = pd.to_datetime(df['admission_date'])

In [4]:
def pvi(df):
    id_list = df['eye_id'].unique()
    pvi_list = []
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        pdf.sort_values(by=['admission_date'], inplace=True)
        pdf.dropna(subset=['visual_acuity'], inplace=True)
        visions = pdf['visual_acuity'].to_list()
        pvi_pdf = max(visions) - visions[0]
        pvi_list.append(pvi_pdf)
    return pvi_list

In [5]:
pvi_list = pvi(df)
len(pvi_list)

544

In [9]:
# this function creates a column for a patient with the interval length
def interval_column(pdf):
    '''
    Input: patient dataframe.
    Output: patient dataframe with additional column for interval length.
    '''
    pdf.dropna(subset=['admission_date'], inplace=True)
    pdf['admission_date'] = pd.to_datetime(pdf['admission_date'])
    pdf = pdf.sort_values(by=['admission_date'])
    dates = pdf['admission_date'].reset_index(drop=True)
    intervals = [0]
    for i in range(len(dates)-1):
        initial, current = dates[i], dates[i+1]
        interval = int((current - initial).days / 7)
        intervals.append(interval)
    pdf['Interval'] = intervals
    return pdf.reset_index(drop=True)

def interval_all(df):
    '''
    Input: whole dataframe.
    Output: whole dataframe with additional column for interval length.
    '''
    id_list = df['eye_id'].unique()
    frames = []
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        pdf = interval_column(pdf)
        frames.append(pdf)
    return pd.concat(frames)

# this function groups dataframes by patients on single vs mult drugs
def drug_df_separator(df):
    df = interval_all(df)
    df = df.dropna(subset=['Drug'])
    df = df[df['Drug'] != 'nil']
    df = df[df['Drug'] != 'None']
    df = df[df['Drug'] != 'Brolucizumab']
    lucentis, eylea, avastin, mult = [], [], [], []
    drug_list = ['Lucentis', 'Eylea', 'Avastin']
    id_list = df['eye_id'].unique()
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        drugs = pdf['Drug'].unique()
        if len(drugs) == 1:
            if drugs[0] == 'Lucentis':
                lucentis.append(pdf)
            elif drugs[0] == 'Eylea':
                eylea.append(pdf)
            else:
                avastin.append(pdf)
        else:
            mult.append(pdf)
    return pd.concat(lucentis), pd.concat(eylea), pd.concat(avastin), pd.concat(mult)

lucentis_id, eylea_id, avastin_id, mult_id = drug_df_separator(df)
len(lucentis_id), len(eylea_id), len(avastin_id), len(mult_id)

(846, 848, 41, 4897)

In [77]:
dfs = [luc_eyl, eyl_luc]
pvi_lists = []
for df in dfs:
    pvi_list = pvi(df)
    pvi_lists.append(pvi_list)

In [78]:
for lst in pvi_lists:
    ci = stats.t.interval(alpha=0.95, df=len(lst)-1, loc=np.mean(lst), scale=stats.sem(lst))
    std = np.mean(lst) - ci[0]
    print("The mean is {}, and the std is {}.".format(np.mean(lst), std))

The mean is 37.88194444444444, and the std is 5.691318565191814.
The mean is 15.936507936507937, and the std is 5.291657625285801.


In [80]:
# Mann-Whitney U test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu
# compare samples
stat, p = mannwhitneyu(pvi_lists[0], pvi_lists[1])

In [81]:
p

0.00047882694368759775

## Time to peak visual improvement

In [26]:
df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dr2_boronia_2016.csv')
df['admission_date'] = pd.to_datetime(df['admission_date'])

In [27]:
def tpvi(df):
    id_list = df['eye_id'].unique()
    tpvi_list = []
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        pdf.sort_values(by=['admission_date'], inplace=True)
        dates = pdf['admission_date'].to_list()
        initial_date = dates[0]
        pdf.dropna(subset=['visual_acuity'], inplace=True)
        visions = pdf['visual_acuity'].to_list()
        ndf = pdf[pdf['visual_acuity'] == max(visions)]
        new_dates = ndf['admission_date'].to_list()
        time = (new_dates[0] - initial_date).days
        tpvi_list.append(time)
    return tpvi_list

In [28]:
tpvi_list = tpvi(df)

In [29]:
np.mean(tpvi_list)

207.2702205882353

In [74]:
dfs = [luc_eyl, eyl_luc]
tpvi_lists = []
for df in dfs:
    tpvi_list = tpvi(df)
    tpvi_lists.append(tpvi_list)

In [75]:
for lst in tpvi_lists:
    ci = stats.t.interval(alpha=0.95, df=len(lst)-1, loc=np.mean(lst), scale=stats.sem(lst))
    std = np.mean(lst) - ci[0]
    print("The mean is {}, and the std is {}.".format(np.mean(lst), std))

The mean is 290.8611111111111, and the std is 46.61831430900676.
The mean is 243.79365079365078, and the std is 62.63150341307809.


In [76]:
stat, p = mannwhitneyu(tpvi_lists[0], tpvi_lists[1])
p

0.15648849605385323

## Segment patients by improved, not improved

In [37]:
df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dr2_boronia_2016.csv')
df['admission_date'] = pd.to_datetime(df['admission_date'])

In [19]:
# this function groups dataframes by patients on single vs mult drugs
def drug_df_separator(df):
    improved, not_improved = [], []
    id_list = df['eye_id'].unique()
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        pdf.dropna(subset=['visual_acuity'], inplace=True)
        pdf.sort_values(by=['admission_date'], inplace=True)
        vision = pdf['visual_acuity'].to_list()
        if max(vision) == vision[0]:
            not_improved.append(pdf)
        else:
            improved.append(pdf)
    return pd.concat(improved), pd.concat(not_improved)

improved, not_improved = drug_df_separator(df)

In [46]:
dfs = [lucentis_id, eylea_id, avastin_id, mult_id]
pvi_lists = []
for df in dfs:
    pvi_list = pvi(df)
    pvi_lists.append(pvi_list)
    
for lst in pvi_lists:
    print(np.mean(lst))

20.133333333333333
15.523809523809524
0.0
31.28708133971292


## Vision loss from peak

In [33]:
def vlp(df):
    id_list = df['eye_id'].unique()
    vlp_list = []
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        pdf.sort_values(by=['admission_date'], inplace=True)
        pdf.dropna(subset=['visual_acuity'], inplace=True)
        visions = pdf['visual_acuity'].to_list()
        recent = visions[-1]
        max_vision = max(visions)
        vlp_list.append(max_vision - recent)
    return vlp_list

In [34]:
vlp_list = vlp(df)
np.mean(vlp_list)

9.913875598086124

In [72]:
dfs = [luc_eyl, eyl_luc]
vlp_lists = []
for df in dfs:
    vlp_list = vlp(df)
    vlp_lists.append(vlp_list)
    
for lst in vlp_lists:
    ci = stats.t.interval(alpha=0.95, df=len(lst)-1, loc=np.mean(lst), scale=stats.sem(lst))
    std = np.mean(lst) - ci[0]
    print("The mean is {}, and the std is {}.".format(np.mean(lst), std))

The mean is 9.48611111111111, and the std is 1.960788003402742.
The mean is 11.142857142857142, and the std is 2.8098609883524865.


In [73]:
stat, p = mannwhitneyu(vlp_lists[0], vlp_lists[1])
p

0.05462183905364518

## Overall visual change

In [51]:
def ovc(df):
    id_list = df['eye_id'].unique()
    ovc_list = []
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        pdf.sort_values(by=['admission_date'], inplace=True)
        pdf.dropna(subset=['visual_acuity'], inplace=True)
        visions = pdf['visual_acuity'].to_list()
        recent = visions[-1]
        initial = visions[0]
        ovc_list.append(initial - recent)
    return ovc_list

In [65]:
dfs = [luc_eyl, eyl_luc]
ovc_lists = []
for df in dfs:
    ovc_list = ovc(df)
    ovc_lists.append(ovc_list)
    
for lst in ovc_lists:
    ci = stats.t.interval(alpha=0.95, df=len(lst)-1, loc=np.mean(lst), scale=stats.sem(lst))
    std = np.mean(lst) - ci[0]
    print("The mean is {}, and the std is {}.".format(np.mean(lst), std))

The mean is -28.395833333333332, and the std is 6.073959754778723.
The mean is -4.7936507936507935, and the std is 5.318709291096716.


In [58]:
stat, p = mannwhitneyu(pvi_lists[1], pvi_lists[3])
p

8.9285078409847e-06

In [83]:
15.93 - 11.14

4.789999999999999

## Significant switches

In [62]:
# this function groups dataframes by patients on single vs mult drugs
def switch_df_separator(df):
    df = interval_all(df)
    df = df.dropna(subset=['Drug'])
    df = df[df['Drug'] != 'nil']
    df = df[df['Drug'] != 'None']
    df = df[df['Drug'] != 'Brolucizumab']
    df = df[df['Drug'] != 'Avastin']
    luc_eyl, eyl_luc = [], []
    drug_list = ['Lucentis', 'Eylea']
    id_list = df['eye_id'].unique()
    for eye in id_list:
        pdf = df[df['eye_id'] == eye]
        drugs = pdf['Drug'].unique()
        if len(drugs) > 1:
            if drugs[0] == 'Lucentis' and drugs[1] == 'Eylea':
                luc_eyl.append(pdf)
            elif drugs[0] == 'Eylea' and drugs[1] == 'Lucentis':
                eyl_luc.append(pdf)
        else:
            pass
    return pd.concat(luc_eyl), pd.concat(eyl_luc)

luc_eyl, eyl_luc = switch_df_separator(df)
len(luc_eyl), len(eyl_luc)

(2967, 1881)

In [63]:
np.mean(luc_eyl['visual_acuity'])

62.549713515335355

In [64]:
np.mean(eyl_luc['visual_acuity'])

63.47900053163211

In [71]:
len(luc_eyl['eye_id'].unique())

144

In [69]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'eye_id', 'next_visit_date',
       'is_submacular_blood_present', 'created_at', 'updated_at',
       'is_floaters_present', 'is_irf_present', 'is_irf_gt_initiation',
       'is_srf_present', 'is_srf_gt_initiation', 'is_ped_gt_last_visit',
       'is_ped_gt_initiation', 'is_submacular_blood_new',
       'is_vision_worse_than_last_visit', 'next_interval_in_weeks', 'drug_id',
       'next_drug_id', 'next_recommendation_eye_state', 'admission_date',
       'message', 'has_submacular_blood_resolved', 'should_extend',
       'doctor_visit_type', 'doctor_visits_id', 'irf_srf_ped_lt_previous_drug',
       'resume_treatment', 'is_atypical', 'status_from_silverpond',
       'silverpond_original_image_url', 'silverpond_overlay_image_url',
       'is_oct_correct', 'is_irf_gt_initiation_prediction',
       'is_srf_gt_initiation_prediction', 'is_recommendation_changed',
       'Recommended drug', 'Recommended next drug',
       'Recommended next interval',

## Visual acuity testing class

In [88]:
df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dr2_boronia_2016.csv')
df['admission_date'] = pd.to_datetime(df['admission_date'])

In [89]:
df = interval_all(df)
df.to_csv('/home/jupyter/charliemacuject/pharma_reports/data/dr2_boronia_2016.csv')

In [105]:
class Vision:
    
    # df_list can be either drugs, improved, or switched
    def __init__(self, df_list):
        self.df_list = df_list
        
    def get_df(self, drop_drug_na=False):
        df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dr2_boronia_2016.csv')
        df['admission_date'] = pd.to_datetime(df['admission_date'])
        if drop_drug_na:
            df = df.dropna(subset=['Drug'])
            df = df[df['Drug'] != 'nil']
            df = df[df['Drug'] != 'None']
            df = df[df['Drug'] != 'Brolucizumab']
        return df
        
    def get_dataframes(self):
        if self.df_list == 'drugs':
            lucentis_df, eylea_df, avastin_df, mult_df = self.drug_df_separator()
            return [lucentis_df, eylea_df, avastin_df, mult_df]
        elif self.df_list == 'switched':
            luc_eyl, eyl_luc = self.switch_df_separator()
            luc_eyl
            return [luc_eyl, eyl_luc]
        elif self.df_list == 'improved':
            improved, not_improved, overall = self.improved_df_separator()
            return [improved, not_improved, overall]
            
    # groups dataframes by patients who improved vs didn't
    def improved_df_separator(self):
        df = self.get_df()
        improved, not_improved = [], []
        id_list = df['eye_id'].unique()
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            pdf.sort_values(by=['admission_date'], inplace=True)
            vision = pdf['visual_acuity'].to_list()
            if max(vision) == vision[0]:
                not_improved.append(pdf)
            else:
                improved.append(pdf)
        return pd.concat(improved), pd.concat(not_improved), df
            
    
    # this function groups dataframes by patients with specific switches
    def switch_df_separator(self):
        df = self.get_df(drop_drug_na=True)
        df = df[df['Drug'] != 'Avastin']
        luc_eyl, eyl_luc = [], []
        drug_list = ['Lucentis', 'Eylea']
        id_list = df['eye_id'].unique()
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            drugs = pdf['Drug'].unique()
            if len(drugs) > 1:
                if drugs[0] == 'Lucentis' and drugs[1] == 'Eylea':
                    luc_eyl.append(pdf)
                elif drugs[0] == 'Eylea' and drugs[1] == 'Lucentis':
                    eyl_luc.append(pdf)
            else:
                pass
        return pd.concat(luc_eyl), pd.concat(eyl_luc)
    
    # this function groups dataframes by patients on single vs mult drugs
    def drug_df_separator(self):
        df = self.get_df(drop_drug_na=True)
        df = df.dropna(subset=['Drug'])
        df = df[df['Drug'] != 'nil']
        df = df[df['Drug'] != 'None']
        df = df[df['Drug'] != 'Brolucizumab']
        lucentis, eylea, avastin, mult = [], [], [], []
        drug_list = ['Lucentis', 'Eylea', 'Avastin']
        id_list = df['eye_id'].unique()
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            drugs = pdf['Drug'].unique()
            if len(drugs) == 1:
                if drugs[0] == 'Lucentis':
                    lucentis.append(pdf)
                elif drugs[0] == 'Eylea':
                    eylea.append(pdf)
                else:
                    avastin.append(pdf)
            else:
                mult.append(pdf)
        return pd.concat(lucentis), pd.concat(eylea), pd.concat(avastin), pd.concat(mult)
    
    def pvi(df):
        id_list = df['eye_id'].unique()
        pvi_list = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            visions = pdf['visual_acuity'].to_list()
            pvi_pdf = max(visions) - visions[0]
            pvi_list.append(pvi_pdf)
        return pvi_list
    
    def tpvi(df):
        id_list = df['eye_id'].unique()
        tpvi_list = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            dates = pdf['admission_date'].to_list()
            initial_date = dates[0]
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            visions = pdf['visual_acuity'].to_list()
            ndf = pdf[pdf['visual_acuity'] == max(visions)]
            new_dates = ndf['admission_date'].to_list()
            time = (new_dates[0] - initial_date).days
            tpvi_list.append(time)
        return tpvi_list
    
    def ovc(df):
        id_list = df['eye_id'].unique()
        ovc_list = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            visions = pdf['visual_acuity'].to_list()
            recent = visions[-1]
            initial = visions[0]
            ovc_list.append(initial - recent)
        return ovc_list
    
    def vlp(df):
        id_list = df['eye_id'].unique()
        vlp_list = []
        for eye in id_list:
            pdf = df[df['eye_id'] == eye]
            pdf.sort_values(by=['admission_date'], inplace=True)
            pdf.dropna(subset=['visual_acuity'], inplace=True)
            visions = pdf['visual_acuity'].to_list()
            recent = visions[-1]
            max_vision = max(visions)
            vlp_list.append(max_vision - recent)
        return vlp_list
    
    def list_results(self, df):
        pvi_list, tpvi_list, vlp_list, ovc_list = pvi(df), tpvi(df), vlp(df), ovc(df)
        return np.mean(pvi_list), np.mean(tpvi_list), np.mean(vlp_list), np.mean(ovc_list)

    def print_results(self):
        dfs = self.get_dataframes()
        lst = []
        if self.df_list == 'Drugs':
            print("Displaying results for Lucentis, Eylea, and Multiple Drugs.")
        elif self.df_list == 'Improved':
            print("Displaying results for Improved, Not_Improved and Overall.")
        else:
            print("Displaying results for patients who switched from Lucentis to Eylea, and Eylea to Lucentis.")
        for dataframe in dfs:
            pvi, tpvi, vlp, ovc = self.list_results(dataframe)
            print("  ")
            print("PVI = {}".forma)
            print("TPVI = {} days")
            print(VLP = {}, OCV = {}".format(pvi, tpvi, vlp, ovc))
            print("  ")
        return lst

In [106]:
improvements = Vision("improved")
improvements.print_results()

Displaying results for patients who switched from Lucentis to Eylea, and Eylea to Lucentis.
PVI = 24.67989417989418, TPVI = 298.2936507936508 days, VLP = 9.542328042328043, OCV = -15.137566137566138
  
PVI = 0.0, TPVI = 0.0 days, VLP = 8.144578313253012, OCV = 8.144578313253012
  
PVI = 17.14889705882353, TPVI = 207.2702205882353 days, VLP = 9.115808823529411, OCV = -8.033088235294118
  


[]